In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,380 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,037 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

def setup_driver():
    """Настройка headless Chrome"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def parse_announcements_table(driver):
    """
    Парсит таблицу с объявлениями
    """
    data = []
    rows = driver.find_elements(By.CSS_SELECTOR, "table.table-bordered tbody tr")
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) < 10:
            continue  # пропускаем строки без данных

        try:
            number = cols[0].text.strip()
            organizer = cols[1].text.strip()
            title = cols[2].text.strip()
            method = cols[3].text.strip()
            item_type = cols[4].text.strip()
            date_start = cols[5].text.strip()
            date_end = cols[6].text.strip()
            lot_count = cols[7].text.strip()
            amount = cols[8].text.strip()
            status = cols[9].text.strip()

            data.append([
                number,
                organizer,
                title,
                method,
                item_type,
                date_start,
                date_end,
                lot_count,
                amount,
                status
            ])
        except Exception as e:
            print("Ошибка парсинга строки:", e)
            continue

    return data

def main():
    url = "https://fms.ecc.kz/ru/searchanno"
    driver = setup_driver()
    driver.get(url)
    time.sleep(2)

    all_data = []

    while True:
        print(f"Собираем данные со страницы: {driver.current_url}")
        table_data = parse_announcements_table(driver)
        all_data.extend(table_data)

        # Переход к следующей странице
        try:
            next_btn = driver.find_element(By.XPATH, "//ul[@class='pagination']//a[text()='>']")
            next_btn.click()
            time.sleep(2)
        except (NoSuchElementException, ElementNotInteractableException):
            print("Нет кнопки '>' — конец пагинации.")
            break

    driver.quit()

    df = pd.DataFrame(all_data, columns=[
        "№",
        "Организатор",
        "Название объявления",
        "Способ закупки",
        "Вид предмета закупки",
        "Дата начала приема заявок",
        "Дата окончания приема заявок",
        "Кол-во лотов",
        "Сумма объявления",
        "Статус"
    ])

    df.to_csv("announcements.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Сохранено {len(df)} записей в announcements.csv")

    return df

if __name__ == "__main__":
    df = main()

    # Для отображения в Jupyter / Colab:
    try:
        from IPython.display import display
        display(df)
    except:
        pass

Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=2
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=3
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=4
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=5
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=6
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=7
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=8
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=9
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=10
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=11
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=12
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=13
Собираем данные со страницы: https://fms.ecc.kz/ru/searchanno?&page=14
Собираем данные со стра

,№,Организатор,Название объявления,Способ закупки,Вид предмета закупки,Дата начала приема заявок,Дата окончания приема заявок,Кол-во лотов,Сумма объявления,Статус
0,497356-1,ФИЛИАЛ ПО КАРАГАНДИНСКОЙ ОБЛАСТИ НЕКОММЕРЧЕСКО...,[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Закуп медицинских услуг,Услуга,2025-03-20 18:00:00,2025-03-26 18:00:00,8,1 733 074 760.53,Опубликовано (прием заявок)
1,497336-1,Филиал по Акмолинской области некоммерческого ...,[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Закуп медицинских услуг,Услуга,2025-03-19 19:00:00,2025-03-26 10:00:00,4,883 026 017.94,Опубликовано (прием заявок)
2,497316-1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств на 2025 год\n2025 ...,Тендер,Товар,2025-03-19 09:00:00,2025-03-31 10:00:00,2,42 606 533.12,Опубликовано (прием заявок)
3,497296-1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп медицинских изделий на 2025 год по Долго...,Из одного источника,Товар,2025-03-19 10:00:00,2025-03-31 10:00:00,1,479 220.00,Завершено
4,497216-1,"""ӘЛЕУМЕТТІК МЕДИЦИНАЛЫҚ САҚТАНДЫРУ ҚОРЫ"" КОММЕ...",[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Закуп медицинских услуг,Услуга,2025-03-13 18:10:00,2025-03-14 18:30:00,1,172 008 324.00,Завершено
...,...,...,...,...,...,...,...,...,...,...
3782,361800-1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Двухэтапный тендер,Товар,2018-11-15 16:48:32,2018-11-20 16:48:32,15,1 359 696 373.73,Завершено
3783,361740-1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Из одного источника,Товар,2018-10-09 09:00:00,2018-10-12 09:00:00,1,10 118 727.36,Завершено
3784,361720-1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Двухэтапный тендер,Товар,2018-10-29 09:00:34,2018-11-01 09:00:34,268,58 732 696 003.07,Завершено
3785,361700-1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Двухэтапный тендер,Товар,2018-10-04 09:00:17,2018-10-09 09:00:17,300,39 435 113 001.95,Завершено
